# Grounded SAM 2 Tracking Demo (Colab)

This notebook mirrors `grounded_sam2_tracking_demo.py` so you can run the tracking pipeline end-to-end on Google Colab.  Use a **GPU runtime** (`Runtime → Change runtime type → GPU`) for the best experience.

In [ ]:
import torch, platform
print(f"Python: {platform.python_version()}")
print(f"Torch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name} | SM {gpu.major}.{gpu.minor} | {gpu.total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No CUDA device detected. Switch the Colab runtime to GPU for faster inference.")

In [ ]:
# Install the packages that the tracking demo depends on.
%%capture
!pip install --upgrade pip
!pip install supervision==0.17.1 transformers accelerate safetensors opencv-python pycocotools hydra-core iopath timm einops huggingface-hub

In [ ]:
# Clone the Grounded SAM 2 repository (change the URL if you are working from a fork).
import os

REPO_URL = "https://github.com/continue-research/Grounded-SAM-2.git"  # @title Repository URL
REPO_DIR = os.path.basename(REPO_URL.rstrip("/").replace(".git", ""))

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

%cd {REPO_DIR}

# Ensure the package search path includes the repo root so local imports work.
import sys
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

In [ ]:
# Download the SAM2 and Grounding DINO checkpoints used by the demo.
!bash checkpoints/download_ckpts.sh large
!bash gdino_checkpoints/download_ckpts.sh swint_ogc

In [ ]:
# Run the tracking demo. Results will be written to ./tracking_results and a video in the repo root.
import os

# Force CUDA when available; fall back to CPU on other runtimes (e.g., Apple M-series).
if torch.cuda.is_available():
    os.environ["SAM2_DEVICE"] = "cuda"
else:
    os.environ["SAM2_DEVICE"] = "cpu"

!python grounded_sam2_tracking_demo.py